In [44]:
# Simple thermostat API using FastAPI
# importing libraries
from fastapi import FastAPI, Depends
import uvicorn
import nest_asyncio

app = FastAPI()


In [45]:
# Defining the thermostat class
class Thermostat:
    def __init__(self,
                 name: str,
                 temperature: int,
                 humidity: int,
                 mode: str,
                 kb_id: str,
                 ):
        self.name = name
        self.temperature = temperature
        self.humidity = humidity
        self.mode = mode
        self.kb_id = kb_id


In [46]:
# Creating a thermostat object
thermostat = Thermostat(name="Thermostat",
                        temperature=20,
                        humidity=50,
                        mode="cool",
                        kb_id="http://127.0.0.1:8000/thermostat",
                        )

In [47]:
thermostat.__dict__

{'name': 'Thermostat',
 'temperature': 20,
 'humidity': 50,
 'mode': 'cool',
 'kb_id': 'http://127.0.0.1:8000/thermostat'}

In [48]:
# Defining the API endpoints
@app.get("/")
async def root():
    return {"message": "Welcome to the thermostat API"}


@app.get("/thermostat")
async def get_thermostat():
    return thermostat.__dict__


@app.get("/thermostat/name")
async def get_name():
    return thermostat.name


@app.get("/thermostat/temperature")
async def get_temperature():
    return thermostat.temperature


@app.get("/thermostat/humidity")
async def get_humidity():
    return thermostat.humidity


@app.get("/thermostat/mode")
async def get_mode():
    return thermostat.mode


@app.get("/thermostat/kb_id")
async def get_kb_id():
    return thermostat.kb_id

@app.put("/thermostat")
async def set_thermostat(commons: Thermostat = Depends()):
    thermostat.name = commons.name
    thermostat.temperature = commons.temperature
    thermostat.humidity = commons.humidity
    thermostat.mode = commons.mode
    thermostat.kb_id = commons.kb_id

    return thermostat.__dict__

@app.put("/thermostat/temperature")
async def set_temperature(temperature: int):
    thermostat.temperature = temperature
    return thermostat.temperature


@app.put("/thermostat/humidity")
async def set_humidity(humidity: int):
    thermostat.humidity = humidity
    return thermostat.humidity


@app.put("/thermostat/mode")
async def set_mode(mode: str):
    thermostat.mode = mode
    return thermostat.mode


@app.put("/thermostat/name")
async def set_name(name: str):
    thermostat.name = name
    return thermostat.name

@app.put("/thermostat/kb_id")
async def set_kb_id(kb_id: str):
    thermostat.kb_id = kb_id
    return thermostat.kb_id


In [49]:
# Defining constants

# Knowledge engine REST API URL
URL = "http://127.0.0.1:8280/rest"

# Knowledge engine REST API headers
HEADERS = {
    'Content-Type': 'application/json',
    'Knowledge-Base-Id': thermostat.kb_id,
}


In [50]:
# register API in the knowledge engine
import httpx

# Creating a smart connector
def create_smart_connector():
    # Smart connector data
    sc_data = {
        "knowledgeBaseId": thermostat.kb_id,
        "knowledgeBaseName": thermostat.name,
        "knowledgeBaseDescription": "Thermostat API v0.0.1 for the Smart Home",
        "reasonerEnabled": "false",
    }

    # Registering the smart connector via the /sc endpoint
    response = httpx.post(URL + "/sc", headers=HEADERS, json=sc_data)

    print(response.status_code)
    print(response.text)

create_smart_connector()

200



In [51]:
# check if the smart connector is registered
def check_smart_connector():
    response = httpx.get(URL + "/sc", headers=HEADERS)
    print(response.status_code)
    print(response.text)

check_smart_connector()


200
[{"knowledgeBaseId":"http://127.0.0.1:8000/thermostat","knowledgeBaseName":"Thermostat","knowledgeBaseDescription":"Thermostat API v0.0.1 for the Smart Home","reasonerEnabled":false}]


In [52]:
# register an ANSWER Knowledge Interaction with the smart connector
def register_answer_ki():
    answer_ki_data = {
        "knowledgeInteractionType": "AnswerKnowledgeInteraction",
        "prefixes": {"rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
                     "saref": "https://w3id.org/saref#",
                     },

        "graphPattern": '''?meas rdf:type saref:Measurement .
                            ?meas saref:hasValue ?temp .
                            ?meas saref:isMeasuredIn saref:TemperatureUnit .
                            ?meas saref:hasTimestamp ?timestamp .
                            ?meas saref:isMeasurementOf ?room_id .
                            ?meas saref:relatesToProperty saref:Temperature .
                            ?meas saref:measurementMadeBy ?device_id .'''
    }

    response = httpx.post(URL + "/sc/ki", headers=HEADERS, json=answer_ki_data)
    print(response.status_code)
    print(response.text)

register_answer_ki()


200
{"knowledgeInteractionId":"http://127.0.0.1:8000/thermostat/interaction/f2f54847-55be-4102-9c3e-5ae7a08ceba8"}


In [53]:
# check if the Knowledge Interaction is registered
def list_all_kis():
    response = httpx.get(URL + "/sc/ki", headers=HEADERS)
    return response.json()
list_all_kis()

[{'knowledgeInteractionId': 'http://127.0.0.1:8000/thermostat/interaction/f2f54847-55be-4102-9c3e-5ae7a08ceba8',
  'knowledgeInteractionType': 'AnswerKnowledgeInteraction',
  'communicativeAct': {'requiredPurposes': ['https://w3id.org/knowledge-engine/InformPurpose'],
   'satisfiedPurposes': ['https://w3id.org/knowledge-engine/InformPurpose']},
  'graphPattern': '?meas <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://w3id.org/saref#Measurement> . ?meas <https://w3id.org/saref#hasValue> ?temp . ?meas <https://w3id.org/saref#isMeasuredIn> <https://w3id.org/saref#TemperatureUnit> . ?meas <https://w3id.org/saref#hasTimestamp> ?timestamp . ?meas <https://w3id.org/saref#isMeasurementOf> ?room_id . ?meas <https://w3id.org/saref#relatesToProperty> <https://w3id.org/saref#Temperature> . ?meas <https://w3id.org/saref#measurementMadeBy> ?device_id . '}]

In [54]:
# Start waiting for a handle request for the given Knowledge Base Id.
def handle_and_answer():
    response = httpx.get(URL + "/sc/handle", headers=HEADERS, timeout=None)
    print(response.status_code)
    print(response.text)
    ki_id = list_all_kis()[-1]['knowledgeInteractionId']
    answer_headers = {
         'Content-Type': 'application/json',
         'Knowledge-Base-Id': thermostat.kb_id,
         'Knowledge-Interaction-Id': ki_id,
    }
    handleRequestId = response.json()["handleRequestId"]
    # deviceName = response.json()["bindingSet"][0]["deviceName"]
    answerBindingSet = [] #???

    # Answering the handle request
    answer_data = {
        "handleRequestId": handleRequestId,
        "bindingSet": answerBindingSet,
    }

    response = httpx.post(URL + "/sc/handle", headers=answer_headers, json=answer_data)
    print(response.status_code)
    print(response.text)
handle_and_answer()

200
{"knowledgeInteractionId":"http://127.0.0.1:8000/thermostat/interaction/f2f54847-55be-4102-9c3e-5ae7a08ceba8","handleRequestId":1,"bindingSet":[],"requestingKnowledgeBaseId":"http://127.0.0.1:8000/subscriber"}
200



In [ ]:
# Running the API
if __name__ == "__main__":
    nest_asyncio.apply()
    #uvicorn.run(app, host="0.0.0.0", port=8001)
    uvicorn.run(app)